# Talking to GPT-3.5-Turbo

In this notebook, we will call the openai api and use the advanced llm gpt-3.5-turbo to do sentiment analysis on our rap-dataset. This is a simple but powerful showcase of what new possibilities llm can offer. Rap texts are notoriously hard for automatic analysis, but gpt models above the third iteration yield impressive results.

Results are sample-wise verified by me.

In [113]:
import numpy as np
import openai
import requests
import json
import pandas as pd
import time
import re
import os

In [412]:
# Getting the private authorization key from file
with open('gpt-auth-key.txt') as f:
    AUTH_KEY = f.read().strip('\n')
#AUTH_KEY

In [30]:
# First test

api_endpoint = 'https://api.openai.com/v1/completions'

params =  {"model": "text-davinci-003",
          "prompt": "I'm flying and have feathers, what am I?",
          "max_tokens": 7,
          "temperature": 0,
            }
response = requests.post(api_endpoint, headers={'Authorization': 'Bearer '+AUTH_KEY}, json=params)#, data=json.dumps(params))
response.json()

{'id': 'cmpl-6w6uEGABAbtfzRycNEZuy2rGBKbdA',
 'object': 'text_completion',
 'created': 1679307590,
 'model': 'text-davinci-003',
 'choices': [{'text': '\n\nYou are a bird.',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 11, 'completion_tokens': 7, 'total_tokens': 18}}

### Further Tests

Following are two tests with randomly chosen rap texts, to get a feeling for the gpt-models and prompt wording.

In [47]:
text = "[Part 1: G.G.B]\
Ich bin ziemlich drauf\
Trink\' die Flasche auf Ex aus\
Was für Koka in der Nase? Ich bin selbstbewusst von Haus\
Scheiß auf Staub (Ja), Diggi, laber mich nicht taub\
Heute machst du alles klar, doch morgen bist du wieder raus oder down\
Ich drehe Runden in der Town\
Rote Lichter, Block ist grau\
An der Hand \'ne schöne Frau, Augen blau\
Ich sag: „Komm zu mir nach Haus!“\
Und der Regen prasselt auf die Kappe direkt oben drauf\
Es ist laut\
Hab\' das Fenster offen, brauch\' Ruhe\
Denn der Kopf am hammеrn und ich sitze auf der Couch\
Hör\' den Sound (Yеs, Mann)\
Während ich an meiner Kippe zieh\'\
Die Lippen leicht zusammengepresst, wenn ich auf meinem Kissen lieg\'\
\
[Hook: Kwam.E & Tom Hengst]\
Schreibe auf, was ich hin und wieder seh\', ahnma\'\
Drei Stiche gab der Kurde dem Albaner\
What goes around, comes around, achte auf dein Karma\
Um die Ecke spielt der Teufel Streiche, wieder Drama\
Izza, bloodclot, Benji und der Tommy H, Mann\
(Smoke noch \'n Jib) Pinke Pitaya (Yeah, huh, huh)\
Rauchzeichen wie ein Indiana\
(Cold, Eisberg) Wie der Fujiyama\
\
[Part 2: Tom Hengst]\
Yeah, yeah, ah\
Laufe aus der Tür, links ein dicker Benz (Dicker Benz)\
Aber nicht meiner, rechts ein Junkie, sucht sein\'n letzten Cent\
Kapuze auf, blende aus, was mich belastet (Ja)\
Zwei Uhr dreiundzwanzig nachts und der Teufel schiebt die Nachtschicht\
Ein paar Jungs sind dicht im Schacht, nebeln den Schacht dicht (Ah)\
Im Schatten des Lichts schimmern weiße Kapseln aus Plastik\
Nasen zieh\'n, hastig, McFit-Karten verwandeln Stein zu Staub (Ja)\
Atme ein und aus, zum runterkommen weißer Rauch\
Sieben Grad plus Regen, Mann, die Schritte werden schneller (Ah)\
Für die Jacke knapp acht Scheine, aber Nudeln auf dem Teller\
(Ja, ah, ja)\
Blaue Lichter scheinen heller als das Offen-Schild der Spielo\
(Geh\' ich rein oder nicht?) Wie nach gepackt werden mit Kilos\
\
[Hook: Kwam.E]\
Schreibe auf, was ich hin und wieder seh\', ahnma\' (Ja)\
Drei Stiche gab der Kurde dem Albaner\
What goes around, comes around, achte auf dein Karma\
Um die Ecke spielt der Teufel Streiche, wieder Drama\
Izza, bloodclot, Benji und der Tommy H, Mann\
(Smoke noch \'n Jib) Pinke Pitaya (Huh, huh)\
Rauchzeichen wie ein Indiana\
(Cold, Eisberg, woo) Wie der Fujiyama (Brr)\
[Part 3: Kwam.E]\
Kickback, six Foot in Prada\
Vorname Kwam.E, ja, ich komm\' aus Ghana\
Hörst du an mei\'m Namen, puff\' a lotta Ganja\
Huh, Vision blurry, seh\' Fata Morgana\
(Ah, ich fühl\' mich cozy) In mei\'m Polo-Pyjama\
(Er vercheckt \'ne Kapsel) An den Barmann\
Meine Boys tragen schwarz, vor Bullen am tarnen\
(Locker, lässig) Schiebe keine Panik\
(Wieder nach zwei Uhr) Gönn\' mir Schawarma\
(Ah, ich bin allein) Keiner in der Bahn drin\
Abgehängt, ich versteck\' mich bei \'ner Chick in Barmbek\
Sie sagt nix, rollt mir einen Joint, geht wieder schlafen, ja\
\
[Hook: Kwam.E]\
Schreibe auf, was ich hin und wieder seh\', ahnma\'\
Drei Stiche gab der Kurde dem Albaner\
What goes around, comes around, achte auf dein Karma\
Um die Ecke spielt der Teufel Streiche, wieder Drama\
Izza, bloodclot, Benji und der Tommy H, Mann\
(Smoke noch \'n Jib) Pinke Pitaya (Huh, huh)\
Rauchzeichen wie ein Indiana\
(Cold, Eisberg) Wie der Fujiyama"

In [55]:
api_endpoint = 'https://api.openai.com/v1/chat/completions'

params = {
  "model": "gpt-3.5-turbo",
  "messages": [{"role": "user", "content": "Do a sentiment analysis of following rap text:\n"+text}],
    "max_tokens": 200,
}

response = requests.post(api_endpoint, headers={'Authorization': 'Bearer '+AUTH_KEY}, json=params)
response.json()

{'id': 'chatcmpl-6w7kNzrvqbNHdTW2l544HthozVLZq',
 'object': 'chat.completion',
 'created': 1679310823,
 'model': 'gpt-3.5-turbo-0301',
 'usage': {'prompt_tokens': 1006,
  'completion_tokens': 88,
  'total_tokens': 1094},
 'choices': [{'message': {'role': 'assistant',
    'content': '\n\nThe sentiment of this rap text is generally negative, as the lyrics depict a scene of drug use, violence, and dangerous situations. The use of profanity and aggressive language add to the negative tone of the text. However, there is also a sense of confidence and self-assuredness in the lyrics, particularly in the first part, which may be seen as positive by some. Overall, the sentiment is mixed but leans towards negativity.'},
   'finish_reason': 'stop',
   'index': 0}]}

In [56]:
response.json()['choices'][0]['message']['content']

'\n\nThe sentiment of this rap text is generally negative, as the lyrics depict a scene of drug use, violence, and dangerous situations. The use of profanity and aggressive language add to the negative tone of the text. However, there is also a sense of confidence and self-assuredness in the lyrics, particularly in the first part, which may be seen as positive by some. Overall, the sentiment is mixed but leans towards negativity.'

In [5]:
text = "[Hook]\
Weißwein und Pappbecher\
Mach\' in\'n Weinbergpark noch ein\'n Abstecher\
Weißwein, krass lecker\
Nette Spätis schenken Pappbecher\
Weißwein und Pappbecher\
In den Weinbergpark noch ein\'n Abstecher\
Pappbecher, Eis rein\
Schöne Mädchen schenken Wein ein\
\
[Part]\
Meine schöne Freundin wеckt mich sanft mit leiser Stimme auf\
Fеttes Sorry an die Nachbarn, wir sind immer laut\
Hab\' jetzt \'ne Wohnung über Freunde, Altbau Hinterhaus\
Nie wieder suche ich ein Zimmer über ImmoScout\
Immer noch behindert drauf, aber nicht peinlich (Eh-eh)\
Viele Lines gezogen, aber nicht mit dem Bleistift (Eh-eh)\
Viele meiner Drogen lager\' ich in mei\'m Schreibtisch\
Mein Handgelenk war einsam, aber jetzt ist es eisig (Brr)\
Kaltes Gösser hier, kühles Bierchen da\
Trage schwarze Nike Shorts wie ein Schiedsrichter\
Und ich pfeife auf die Bull\'n, weil ich sie nicht mag\
Der Track und ich sind böse, doch zusamm\'n sind wir ein Liebespaar\
Eine Pappe LSD und ich flieg\' zum Mond\
Seh\' in HD, Augen Mikroskop\
Geh\' mit Bro in die Natur mit einem Kilo Obst\
Deeper Dialog, so high, ich werd\' zum Philosoph\
U-Bahn-Automat, hol\' mir Süßkram\
Bueno und Hanuta teile ich mit mein\'n Brüdern\
Geh\' nicht mehr in die Uni, das ist mir zu mühsam\
Würde wähl\'n geh\'n, aber wen von diesen Lügnern?\
Früher auch schon Schülerausweis gefälscht\
Ausseh\'n wie elf, aber auf Weiß auf Raves\
Uber-Auto bestellt, trotzdem brauch\' ich kaum Geld\
Denn dass die Crew den Kaufpreis aufteilt, ist safe\
Nein, nein (Nein), kein Nine-to-Five und Anzug\
Trage stattdessen Tracksuit (Tracksuit)\
Finde kein\'n Job, doch das nur\
Weil ich kein\'n Stress such\'\
Simir fälscht ein Rezept, holt sich Xans (Xans)\
Macht damit Tec-tec auf Techno-Events (-vents)\
Triff mich im Heckmeck, zieh\' Pep mit der Gang\
Keine Cap, mein Gedächtnis zerfetzt wie die Vans\
\
[Pre-Hook]\
Weißwein und Pappbecher\
Mach\' in\' Weinbergpark noch ein\'n Abstecher\
Schraubverschluss einweih\'n\
Schöne Mädchen schenken Wein ein\
Weißwein und Pappbecher\
Mach\' in\'n Weinbergpark noch ein\'n Abstecher\
Pappbecher, Eis rein\
Schöne Mädchen schenken Wein ein\
\
[Hook]\
Weißwein und Pappbecher\
Mach\' in\'n Weinbergpark noch ein\'n Abstecher\
Weißwein, krass lecker\
Nette Spätis schenken Pappbecher\
Weißwein und Pappbecher\
Mach\' in\'n Weinbergpark noch ein\'n Abstecher\
Pappbecher, Eis rein\
Schöne Mädchen schenken Wein ein"

In [189]:
api_endpoint = 'https://api.openai.com/v1/chat/completions'


params = {
  "model": "gpt-3.5-turbo",
  "messages": [{"role": "user", "content": "\
  Do a sentiment analysis of the following rap text, be precise and concise \
  Try to talk about sentiment and emotions without summarizing much of the content of the text."+text+
  "After the sentiment analysis, write on a newline: EMOTIONS \n \
  Then give 5 to 10 emotions you would attribute to the text.\n"}],
    "max_tokens": 200,
}
try:
 response = requests.post(api_endpoint, headers={'Authorization': 'Bearer '+AUTH_KEY}, json=params, timeout=0.2)
except requests.exceptions.ReadTimeout as e:
    print(e)
response.json()

HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=0.2)


{'id': 'chatcmpl-6xIyA9sMKvOESIKs9mjUIfnJ0rKxf',
 'object': 'chat.completion',
 'created': 1679592290,
 'model': 'gpt-3.5-turbo-0301',
 'usage': {'prompt_tokens': 80, 'completion_tokens': 39, 'total_tokens': 119},
 'choices': [{'message': {'role': 'assistant',
    'content': '\n\nSentiment Analysis: The text expresses a positive sentiment.\n\nEMOTIONS:\n1. Happiness\n2. Contentment\n3. Joyfulness\n4. Optimism\n5. Positivity'},
   'finish_reason': 'stop',
   'index': 0}]}

In [103]:
print(response.json()['choices'][0]['message']['content'])



Sentiment Analysis: The text talks about a carefree and joyful attitude towards life, where the author values leisure activities, partying, and drug consumption over traditional societal norms. 

EMOTIONS:
1. Joy
2. Excitement
3. Rebelliousness
4. Satisfaction
5. Carefree
6. Nostalgia
7. Contentment
8. Fun-loving
9. Eccentricity 
10. Indulgence


In [163]:
sentiment_list = ['happy', 'sad', 'joyful', 'fearful', 'peaceful', 'anger', 'love', 'hate', 'excitement', 'boredom']

In [166]:
', '.join(sentiment_list)

'happy, sad, joyful, fearful, peaceful, anger, love, hate, excitement, boredom'

## Preparing the loop

Now we need to code a loop that goes through our dataset and sends each rap to gpt. We again first test this.

Because I wanted a sentiment analysis and a separate list of 10 emotions describing the test, the loop actually had to be run twice, with different prompts. The first will get the model to output a sentiment analysis, the second will give us the emotion list.

In [413]:
data = pd.read_csv('test_sample.csv')
data['sentiment'] = ['None' for i in range(data.shape[0])]
data.head()

,Unnamed: 0,title,artist,year,views,lyrics,language,sentiment
0,47539,Mein Herz hat sich gewehrt,Laer Xirtam,2017,46,\nDu lässt mich warten\nIch mach enge mit der ...,de,None
1,80895,Misbhv kein fendi,Yakama,2021,46,\n\n\nHab deine Wifey auf meinem Backseat\nSie...,de,None
2,65122,Bucketlist,GOLDFELL,2017,5,Zeitreis! In nem lilanen Schiff\nUnd wenn die ...,de,None
3,89863,AUF DIE ZEITEN DIE MAL WAREN,Tream & treamiboii,2022,384,\n\n\n\n\n\nUnd wir saufen zusammen auf die Ze...,de,None
4,53644,Verrückte Ratten,Nate57 & Telly Tellz,2009,883,"\nMic-Check, was is' das? Red ma' nich', du Kn...",de,None


In [173]:
api_endpoint = 'https://api.openai.com/v1/chat/completions'


#ToDo: improve prompt: use precise words, dont repeat yourself, shorter is better than convoluted...
for idx, row in data.iterrows():
    params = {
              "model": "gpt-3.5-turbo",
              "messages": [{"role": "user", "content": "\
              Do a sentiment analysis of the following rap text, be precise and concise \
              Try to talk about sentiment and emotions without summarizing much of the content of the text."+row.lyrics+
              "After the sentiment analysis, write on a newline: EMOTIONS \n \
              Then give 5 to 10 emotions you would attribute to the text.\n"}],
                "max_tokens": 200,
            }

    
    if data.at[idx, 'sentiment'] == 'None':
        #print("here", data.at[idx, 'sentiment'])
        response = requests.post(api_endpoint, headers={'Authorization': 'Bearer '+AUTH_KEY}, json=params, timeout=30)
        data.at[idx, 'sentiment'] = response.json()['choices'][0]['message']['content']
    
    if idx == 1000:
        print("In iteration 1000, saving...")
        data.to_csv('raps_de_min_10_songs.csv', index=False)

KeyboardInterrupt: 

In [69]:
test_sample

,Unnamed: 0,title,artist,year,views,lyrics,language,sentiment
0,47539,Mein Herz hat sich gewehrt,Laer Xirtam,2017,46,\nDu lässt mich warten\nIch mach enge mit der ...,de,\n\nThe overall sentiment of the rap text is n...
1,80895,Misbhv kein fendi,Yakama,2021,46,\n\n\nHab deine Wifey auf meinem Backseat\nSie...,de,\n\nThe sentiment of the rap text is largely n...
2,65122,Bucketlist,GOLDFELL,2017,5,Zeitreis! In nem lilanen Schiff\nUnd wenn die ...,de,"\n\nOverall, the sentiment of the rap text is ..."
3,89863,AUF DIE ZEITEN DIE MAL WAREN,Tream & treamiboii,2022,384,\n\n\n\n\n\nUnd wir saufen zusammen auf die Ze...,de,\n\nThe sentiment of this rap text is nostalgi...
4,53644,Verrückte Ratten,Nate57 & Telly Tellz,2009,883,"\nMic-Check, was is' das? Red ma' nich', du Kn...",de,\n\nThe sentiment of the rap text is predomina...
5,22947,Liebesbrief,AkA-Dee,2011,670,"\n""Sonst muss ich dir nen Liebesbrief schicken...",de,\n\nThe sentiment of these rap lyrics is highl...
6,64924,In meiner Welt,Slavik,2020,5774,"\n\n\nMorgens wach' ich auf, aber nicht neben ...",de,\n\nThe sentiment of this rap text is negative...
7,22504,Vs. EstA - RR 8tel-Finale - VBT 2012,Dr. Lucs,2012,43,"\nOkay, das ganze VBT kann hören, der Standard...",de,"\n\nOverall, the sentiment in this rap text is..."
8,10515,Intro,Johnny Pepp,2014,101,\nEs ist soweit\nHinterlasse Schutt und Asche\...,de,\n\nThe overall sentiment of the rap text is n...
9,4508,Vs. p0rrr - Vorrunde 2 - VBT 2013,Vitality,2013,86,"\nWie hässlich muss man sein, dass man die Kin...",de,\n\nThe sentiment of the rap text is highly ne...


In [284]:
"""
real_data = pd.read_csv('second_raps_de.csv')
real_data['sentiment'] = ['None' for i in range(real_data.shape[0])]
real_data.head()
"""

,title,artist,year,views,lyrics,language,sentiment
0,Hammerhart,Beginner,1998,20823,"\nOh, na, na\nBambule - Yes, yo\nDies ist Hamb...",de,None
1,Und Was Ist Jetzt?,Curse,2003,9420,\nIch hör' dich jetzt noch wie gestern die Wor...,de,None
2,Zur Erinnerung,Ferris MC,2003,4701,"\nWir lernten uns beide kennen, als wir ins gl...",de,None
3,Popstarz,Ferris MC,2003,474,Kurzes Intro:\n\nYeah Ferris MC und ich hasse ...,de,None
4,Pyramide,Favorite,2011,8648,"\nYeah, yeah, yeah, was geht ab?!\nFavorite 20...",de,None


In [285]:
#real_data.to_csv('sentiment_second_raps_de.csv', index=False)

In [118]:
#real_data = pd.read_csv('second_raps_de.csv')
real_data.head()

,title,artist,year,views,lyrics,language,sentiment
0,Hammerhart,Beginner,1998,20823,"\nOh, na, na\nBambule - Yes, yo\nDies ist Hamb...",de,None
1,Und Was Ist Jetzt?,Curse,2003,9420,\nIch hör' dich jetzt noch wie gestern die Wor...,de,None
2,Zur Erinnerung,Ferris MC,2003,4701,"\nWir lernten uns beide kennen, als wir ins gl...",de,None
3,Popstarz,Ferris MC,2003,474,Kurzes Intro:\n\nYeah Ferris MC und ich hasse ...,de,None
4,Pyramide,Favorite,2011,8648,"\nYeah, yeah, yeah, was geht ab?!\nFavorite 20...",de,None


In [115]:
real_data.title.value_counts()

Intro               21
Outro                8
Mama                 5
Bang Bang            4
Winter               4
                    ..
Geez                 1
Alles dreht sich     1
Am Ende              1
Golden State         1
Sicher               1
Name: title, Length: 3445, dtype: int64

In [ ]:
# This prompt was run twice, with different prompts.

api_endpoint = 'https://api.openai.com/v1/chat/completions'

start = time.time()
print("Starting sentiment analysis...")
iter_time = time.time()
for idx, row in real_data.iterrows():
    #print(row.lyrics)
    reiterate = False
    params = {
              "model": "gpt-3.5-turbo",
              "messages": [{"role": "user", "content": "\
              Do a sentiment analysis of the following rap text, be precise and concise \
              Try to talk about sentiment and emotions without summarizing much of the content of the text."+row.lyrics+
              "After the sentiment analysis, write on a newline: EMOTIONS \n \
              Then give 5 to 10 emotions you would attribute to the text. Do not write an explanation for the emotions, just the emotion itself.\n"}],
                "max_tokens": 200,
            }

    
    if real_data.at[idx, 'sentiment'] == 'None':
        #print("here", data.at[idx, 'sentiment'])
        try:
            response = requests.post(api_endpoint, headers={'Authorization': 'Bearer '+AUTH_KEY}, json=params, timeout=15)
            # Sometimes songs are more than the supported token number, so we simple skip these for now and drop the rows later
            if 'error' in response.json().keys():
                print(response.json())
                continue
                
            real_data.at[idx, 'sentiment'] = response.json()['choices'][0]['message']['content']
        except requests.exceptions.ReadTimeout as e:
            print(e)
            reiterate = True
            while reiterate:
                try:
                    response = requests.post(api_endpoint, headers={'Authorization': 'Bearer '+AUTH_KEY}, json=params, timeout=15)
                    real_data.at[idx, 'sentiment'] = response.json()['choices'][0]['message']['content']
                    print("retry successfull!")
                    reiterate = False
                except requests.exceptions.ReadTimeout as e:
                    print("retrying...", end="")
            
        if idx==1:
            print("Running...")
            print(response.json()['choices'][0]['message'])
        else:
            print(".", end='')
        
    
    if idx%10==0:
        print("\nIn iteration {}, saving...".format(idx))
        end = time.time()
        print("This took {} seconds".format(int(end-iter_time)))
        real_data.to_csv('sentiment_third_raps_de.csv', index=False)
        iter_time = time.time()

end = time.time()    
print("Finished sentiment analysis! This took {} seconds.".format(int(end-start)))
real_data.head()

In [293]:
#real_data.to_csv('sentiment_third_raps_de.csv', index=False)

## Combining both sentiment analysis

Both our gpt queries yielded different results. The first resulted in a detailed sentiment analysis. The second was more precise when it comes to our emotions.
We now want to combine both datasets into one, and clean the results from gpt while we're at it.


### Starting with the sentiment analysis

In [336]:
sentiment_data = pd.read_csv('sentiment_second_raps_de.csv')

In [337]:
combined_data = sentiment_data.copy()
combined_data['emotions'] = ['None' for i in range(real_data.shape[0])]

In [ ]:
pos_of_first_the = []
for idx, row in sentiment_data.iterrows():
    #print(idx)
    text = row['sentiment']#.split(' ')
    #print(text)
    text = ' '.join(text.splitlines()).lower()
    text_list = text.split('emotions')
    
    # cleaning the sentiment
    sentiment = text_list[:-1]
    # If only one sentence is contained in sentiment list, we take it out
    if len(sentiment) == 1:
        sentiment = sentiment[0]
    else:
        sentiment = ' emotions '.join(sentiment)
    start = 0
    for i, word in enumerate(sentiment.split()):
        if word=='the':
            start = i
            break
    sentiment = ' '.join(sentiment.split()[start:])
    print(sentiment)
    print('-----------------------')
    combined_data.at[idx, 'sentiment'] = sentiment
    

### Then, the emotions

In [362]:
emotion_data = pd.read_csv('sentiment_third_raps_de.csv')

In [365]:
emotion_data

,title,artist,year,views,lyrics,language,sentiment
0,Hammerhart,Beginner,1998,20823,"\nOh, na, na\nBambule - Yes, yo\nDies ist Hamb...",de,Sentiment: Positive\n\nEMOTIONS:\n1. Confidenc...
1,Und Was Ist Jetzt?,Curse,2003,9420,\nIch hör' dich jetzt noch wie gestern die Wor...,de,Sentiment: Negative\n\nEMOTIONS:\n1. Betrayal\...
2,Zur Erinnerung,Ferris MC,2003,4701,"\nWir lernten uns beide kennen, als wir ins gl...",de,Sentiment: The text conveys a sense of nostalg...
3,Popstarz,Ferris MC,2003,474,Kurzes Intro:\n\nYeah Ferris MC und ich hasse ...,de,The sentiment of the rap text is negative and ...
4,Pyramide,Favorite,2011,8648,"\nYeah, yeah, yeah, was geht ab?!\nFavorite 20...",de,Sentiment: Negative/Aggressive\n\nEmotions:\n1...
...,...,...,...,...,...,...,...
3636,Habaneros,TIEFBASSKOMMANDO,2022,10,"\nIch hab' ein Colt in meiner Hose, der ist sc...",de,"EMOTIONS:\nAggression, disdain, superiority, d..."
3637,Ein Fall für sich,Manuellsen,2022,87,"\n\n\nLak, shu, diese Zeil'n!\n\n\nIch bin ein...",de,SENTIMENT: The overall sentiment of the text i...
3638,Northern Lights,Samo104,2022,7,"\n\n\n\n\n\nAll eyes on her, ja \nLiebe dein S...",de,Sentiment: The text expresses a positive roman...
3639,Kein Auge zu,Samo104,2022,32,\n\n\nIch mach' schon wieder kein Auge zu\nWei...,de,The sentiment of the text is predominantly neg...


In [ ]:
# real_data is currently emotions data because thats the last call to gpt we made
for idx, row in emotion_data.iterrows():
    #print(idx)
    text = row['sentiment']
    #print(text)
    text = ' '.join(text.splitlines()).lower()
    text_list = text.split('emotions')
    
    #print(text_list)
    # cleaning the emotions
    emotions = text_list[-1]
    #print(emotions)
    emotions = re.split('1|2|3|4|5|6|7|8|9|[10]|,|-|\)|:', emotions)
    emotions = [re.sub('\.|\s|,', '', ele) for ele in emotions]
    #print(emotions)
    new_emots = []
    for ele in emotions:
        if len(ele)>=3 and len(ele)<20:
            new_emots.append(ele)
        elif len(ele)>20:
            print(ele)
    
    #print(new_emots)
    #print('-----------------------')
    combined_data.at[idx, 'emotions'] = new_emots
    #if idx == 4: break

In [398]:
combined_data.to_csv('combined_data.csv', index=False)

## Grading

To be able to better visualize the rap sentiments, we again use gpt to grade the list of emotions on a set of fixed emotions: sadness, happiness, fear, anger, surprise and disgust. This is necessary because the words in our emotion list are really diverse and thus wouldn't give much insight. For visualization and further use-cases, its better of distilling the sentiment down to the core-emotions of humans.

In [399]:
combined_data['grade'] = [-1 for i in range(combined_data.shape[0])]
combined_data

,title,artist,year,views,lyrics,language,sentiment,emotions,grade
0,Hammerhart,Beginner,1998,20823,"\nOh, na, na\nBambule - Yes, yo\nDies ist Hamb...",de,the rap text contains a mix of positivity and ...,"[confidence, passion, excitement, determinatio...",-1
1,Und Was Ist Jetzt?,Curse,2003,9420,\nIch hör' dich jetzt noch wie gestern die Wor...,de,the overall sentiment of the text is one of be...,"[betrayal, heartbreak, anger, desperation, dis...",-1
2,Zur Erinnerung,Ferris MC,2003,4701,"\nWir lernten uns beide kennen, als wir ins gl...",de,the rap text is imbued with feelings of nostal...,"[regret, nostalgia, sadness, remorse, despair,...",-1
3,Popstarz,Ferris MC,2003,474,Kurzes Intro:\n\nYeah Ferris MC und ich hasse ...,de,the rap text expresses hate towards popstars a...,"[hatred, disdain, anger, contempt, frustration...",-1
4,Pyramide,Favorite,2011,8648,"\nYeah, yeah, yeah, was geht ab?!\nFavorite 20...",de,the overall sentiment of the rap text is negat...,"[anger, hostility, contempt, arrogance, violen...",-1
...,...,...,...,...,...,...,...,...,...
3636,Habaneros,TIEFBASSKOMMANDO,2022,10,"\nIch hab' ein Colt in meiner Hose, der ist sc...",de,sentiment: negative/aggressive,"[aggression, disdain, superiority, disrespect,...",-1
3637,Ein Fall für sich,Manuellsen,2022,87,"\n\n\nLak, shu, diese Zeil'n!\n\n\nIch bin ein...",de,the overall sentiment of the text is a mix of ...,"[confusion, comfort, love, lust, belonging, ho...",-1
3638,Northern Lights,Samo104,2022,7,"\n\n\n\n\n\nAll eyes on her, ja \nLiebe dein S...",de,the text expresses a romantic and flirtatious ...,"[desire, infatuation, passion, excitement, lon...",-1
3639,Kein Auge zu,Samo104,2022,32,\n\n\nIch mach' schon wieder kein Auge zu\nWei...,de,"the sentiment of the rap text is negative, wit...","[anxiety, loneliness, despair, chaos, instabil...",-1


In [ ]:
emotions_set = set()
for idx, row in combined_data.iterrows():
    for emo in row.emotions:
        emotions_set.add(emo)

print(len(emotions_set))
emotions_set

In [ ]:
#ToDo: Check if and how many rows in emotions columns are empty!

In [ ]:
api_endpoint = 'https://api.openai.com/v1/chat/completions'

start = time.time()
print("Starting emotion distillation...")
iter_time = time.time()
for idx, row in combined_data.iterrows():
    #print(row.lyrics)
    reiterate = False
    params = {
              "model": "gpt-3.5-turbo",
              "messages": [{"role": "user", "content": "\
              With one word, summarize following words:"+', '.join(row.emotions) + "\' \n \
              Your answer must consist of only one word out of sadness, happiness, fear, anger, surprise and disgust."}],
                "max_tokens": 200,
            }

    
    if combined_data.at[idx, 'grade'] == -1:
        #print("here", data.at[idx, 'sentiment'])
        try:
            response = requests.post(api_endpoint, headers={'Authorization': 'Bearer '+AUTH_KEY}, json=params, timeout=15)
            # Sometimes songs are more than the supported token number, so we simple skip these for now and drop the rows later
            if 'error' in response.json().keys():
                print(response.json())
                if 'Rate limit reached' in response.json()['error']['message']:
                    time.sleep(60)
                    response = requests.post(api_endpoint, headers={'Authorization': 'Bearer '+AUTH_KEY}, json=params, timeout=15)
                    combined_data.at[idx, 'grade'] = response.json()['choices'][0]['message']['content']
                    print("retry successfull!")
                continue
                
            else: combined_data.at[idx, 'grade'] = response.json()['choices'][0]['message']['content']
        except requests.exceptions.ReadTimeout as e:
            print(e)
            time.sleep(60)
            reiterate = True
            while reiterate:
                try:
                    response = requests.post(api_endpoint, headers={'Authorization': 'Bearer '+AUTH_KEY}, json=params, timeout=15)
                    combined_data.at[idx, 'grade'] = response.json()['choices'][0]['message']['content']
                    print("retry successfull!")
                    reiterate = False
                except requests.exceptions.ReadTimeout as e:
                    print("retrying...", end="")
            
        if idx==1:
            print("Running...")
            print(response.json()['choices'][0]['message'])
            #break
        else:
            print(".", end='')
        
    
    if idx%10==0:
        print("\nIn iteration {}, saving...".format(idx))
        end = time.time()
        print("This took {} seconds".format(int(end-iter_time)))
        combined_data.to_csv('combined_data.csv', index=False)
        iter_time = time.time()
        
        #break

end = time.time()    
print("Finished emotion distillation! This took {} seconds.".format(int(end-start)))
combined_data.head()

In [406]:
combined_data.to_csv('combined_data.csv', index=False)

In [421]:
combined_data.head()

,title,artist,year,views,lyrics,language,sentiment,emotions,grade
0,Hammerhart,Beginner,1998,20823,"\nOh, na, na\nBambule - Yes, yo\nDies ist Hamb...",de,the rap text contains a mix of positivity and ...,"[confidence, passion, excitement, determinatio...",Happiness.
1,Und Was Ist Jetzt?,Curse,2003,9420,\nIch hör' dich jetzt noch wie gestern die Wor...,de,the overall sentiment of the text is one of be...,"[betrayal, heartbreak, anger, desperation, dis...",Sadness.
2,Zur Erinnerung,Ferris MC,2003,4701,"\nWir lernten uns beide kennen, als wir ins gl...",de,the rap text is imbued with feelings of nostal...,"[regret, nostalgia, sadness, remorse, despair,...",Sadness.
3,Popstarz,Ferris MC,2003,474,Kurzes Intro:\n\nYeah Ferris MC und ich hasse ...,de,the rap text expresses hate towards popstars a...,"[hatred, disdain, anger, contempt, frustration...",Anger.
4,Pyramide,Favorite,2011,8648,"\nYeah, yeah, yeah, was geht ab?!\nFavorite 20...",de,the overall sentiment of the rap text is negat...,"[anger, hostility, contempt, arrogance, violen...",Disgust.


Thats it for now!